In [ ]:
# Download XLS 

In [4]:
import requests

url = "https://www.screener.in/user/company/export/6598060/"

# Replace this with the actual CSRF token
csrf_token = "A3F65XxcJUem8vdLLBBRXlDCTcTx67Ky0GmfbzzuSvr2QVEJ2D77kdNK93Cze3sE"

# Replace this with the actual session ID
session_id = "gsblyiotrq9442ka3t5vo75c9so0lm63"

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "Referer": "https://www.screener.in/company/PVP/consolidated/",
    "Origin": "https://www.screener.in",
    "Cookie": f"csrftoken={csrf_token}; sessionid={session_id}"
}

# Replace this with the actual payload
payload = {
    "csrfmiddlewaretoken": csrf_token,
    "next": "/company/PVP/consolidated/"
}

# Send the POST request to download the file
response = requests.post(url, headers=headers, data=payload)

# Check if the request was successful (Status Code 200)
if response.status_code == 200:
    # Save the content of the response to a file
    with open("PVP_Ventures.xlsx", "wb") as file:
        file.write(response.content)
    print("File downloaded successfully.")
else:
    print(f"Error: {response.status_code} - {response.text}")


File downloaded successfully.


In [3]:
import os
import requests

def download_excel(company_name, download_path="."):
    url = f"https://www.screener.in/user/company/export/{company_name}/"
    csrf_token = "A3F65XxcJUem8vdLLBBRXlDCTcTx67Ky0GmfbzzuSvr2QVEJ2D77kdNK93Cze3sE"
    session_id = "gsblyiotrq9442ka3t5vo75c9so0lm63"

    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Referer": f"https://www.screener.in/company/{company_name}/consolidated/",
        "Origin": "https://www.screener.in",
        "Cookie": f"csrftoken={csrf_token}; sessionid={session_id}"
    }

    payload = {
        "csrfmiddlewaretoken": csrf_token,
        "next": f"/company/{company_name}/consolidated/"
    }

    response = requests.post(url, headers=headers, data=payload)

    if response.status_code == 200:
        file_path = os.path.join(download_path, f"{company_name}.xlsx")

        with open(file_path, "wb") as file:
            file.write(response.content)

        print(f"File for {company_name} downloaded successfully at {file_path}")
    elif response.status_code == 404:
        print(f"Error 404: {response.text}")
    else:
        print(f"Error for {company_name}: {response.status_code} - {response.text}")

# Example usage:
company_name = "PVP"  # Replace this with the desired company name
download_path = "./data"  # Specify your desired download path
download_excel(company_name, download_path)


Error 404: 
<!DOCTYPE html>
<html lang="en">
  <head>
    <!-- Basic Page Needs -->
    <meta charset="utf-8">
    
    <title>Error 404: Page Not Found - Screener</title>
    
    <meta name="author" content="Mittal Analytics Private Limited">

    <!-- PWA manifest -->
    <meta name="theme-color"
          content="#fff"
          media="(prefers-color-scheme: light)">
    <meta name="theme-color"
          content="#222530"
          media="(prefers-color-scheme: dark)">
    <link rel="manifest" href="/static/manifest.json">

    <!-- favicons -->
    <link rel="apple-touch-icon"
          sizes="180x180"
          href="https://cdn-static.screener.in/favicon/apple-touch-icon.b6e5f24bf7d3.png">
    <link rel="icon"
          type="image/png"
          sizes="32x32"
          href="https://cdn-static.screener.in/favicon/favicon-32x32.00205914303a.png">
    <link rel="icon"
          type="image/png"
          sizes="16x16"
          href="https://cdn-static.screener.in/favicon/favic